In [1]:
import sys

In [3]:
sys.getrefcount

<function sys.getrefcount(object, /)>

In [6]:
sys.getrefcount("import")

28

In [7]:
import cProfile

In [8]:
sys.getrefcount("import")

28

In [9]:
import_str = "import"

In [10]:
sys.getrefcount("import")

29

In [11]:
sys.getrefcount(-5)

9

In [12]:
sys.getrefcount(-6)

2

In [13]:
sys.getrefcount(0)

5983

In [14]:
sys.getrefcount(256), sys.getrefcount(257)

(91, 3)

In [15]:
x = 10 ** 10 + 5

In [16]:
x

10000000005

In [17]:
sys.getrefcount(x)

9

In [22]:
x = 10 ** 10 + 5

print(sys.getrefcount(x))

y = x

print(sys.getrefcount(x))

del x

print(sys.getrefcount(y))

2
3
2


In [25]:
lst = [1, 2, "qwerty"]

print("before", sys.getrefcount(lst))

def count_refs(obj):
    print("count_refs", sys.getrefcount(obj))
    
count_refs(lst)

print("after", sys.getrefcount(lst))

del lst

before 2
count_refs 4
after 2


In [28]:
print("dwd")

dwd


In [31]:
class Person:
    def __del__(self):
        print("Person.__del__", id(self))


pers = Person()
print(f"{id(pers)=}")


print("before", sys.getrefcount(pers))

count_refs(pers)

print("after", sys.getrefcount(pers))

del pers

print("total after")

id(pers)=4367965088
before 2
count_refs 4
after 2
Person.__del__ 4367965088
total after


In [47]:
import gc

In [55]:
class Person:
    def __del__(self):
        print("Person.__del__", id(self))


pers1 = Person()
pers2 = Person()

id_pers1 = id(pers1)
id_pers2 = id(pers2)

print(f"{id(pers1)=}, {id(pers2)=}")

pers1.parent = pers2
pers2.parent = pers1


print("before", sys.getrefcount(pers1), sys.getrefcount(pers2))

count_refs(pers1)
count_refs(pers2)

print("after", sys.getrefcount(pers1), sys.getrefcount(pers2))

del pers1, pers2

print("total after")

print(f"{PyObject.from_address(id_pers1).refcnt=}")
print(f"{PyObject.from_address(id_pers2).refcnt=}")

gc.collect()

print("\nafter gc.collect()")

print(f"{PyObject.from_address(id_pers1).refcnt=}")
print(f"{PyObject.from_address(id_pers2).refcnt=}")

id(pers1)=4367548176, id(pers2)=4367548224
before 3 3
count_refs 5
count_refs 5
after 3 3
total after
PyObject.from_address(id_pers1).refcnt=1
PyObject.from_address(id_pers2).refcnt=1
Person.__del__ 4367548176
Person.__del__ 4367548224

after gc.collect()
PyObject.from_address(id_pers1).refcnt=0
PyObject.from_address(id_pers2).refcnt=0


In [51]:
import ctypes

In [34]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]

In [38]:
lst = [1, 2, "qwerty"]

PyObject.from_address(id(lst)).refcnt

1

In [56]:
import weakref

In [84]:
class Person:
    def __init__(self, age):
        self.age = age
        
    def calc_age(self):
        print("calc_age", self.age)

    def __del__(self):
        print("Person.__del__", self.age)
        
        
def del_handler():
    print("final delete actions")
        

p1 = Person(99)
calc_age_link = weakref.WeakMethod(p1.calc_age)
calc_age_link()()


p2 = Person(101)

weakref.finalize(p1, del_handler)

p1.parent = weakref.ref(p2)
p2.parent = weakref.ref(p1)

print("before sys.getref", sys.getrefcount(p1))

weak = weakref.ref(p1)

print("after sys.getref", sys.getrefcount(p1))

obj = weak()

print("after weak()", sys.getrefcount(p1))

del obj, p1
print("\nafter del")

obj = weak()
print("obj", obj)

obj_meth = calc_age_link()
print("obj_meth", obj)

calc_age 99
before sys.getref 2
after sys.getref 2
after weak() 3
Person.__del__ 99
final delete actions

after del
obj None
obj_meth None


In [66]:
obj = weak()


In [68]:
obj, obj.age

(<__main__.Person at 0x104950f70>, 99)

In [61]:
weak1 = weakref.ref(p1)

In [62]:
weakref.getweakrefcount(p1)

1

In [63]:
weak is weak1

True

In [64]:
sys.getrefcount(p1)

6

In [86]:
sys.getsizeof([1, 2, 3 * 1000])

120

In [88]:
sys.getsizeof({}), sys.getsizeof({1: 11})

(64, 232)

In [89]:
d = {}
for i in range(7):
    print("\nbefore", i, sys.getsizeof(d))
    d[i] = i
    print("after", i, sys.getsizeof(d))


before 0 64
after 0 232

before 1 232
after 1 232

before 2 232
after 2 232

before 3 232
after 3 232

before 4 232
after 4 232

before 5 232
after 5 360

before 6 360
after 6 360


In [90]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        
class PointSlot:
    __slots__ = ("x", "y")

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [91]:
p = Point(10, 20)
ps = PointSlot(30, 40)

In [92]:
p.__dict__

{'x': 10, 'y': 20}

In [93]:
ps.__dict__

AttributeError: 'PointSlot' object has no attribute '__dict__'

In [94]:
Point.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Point.__init__(self, x, y)>,
              '__dict__': <attribute '__dict__' of 'Point' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point' objects>,
              '__doc__': None})

In [95]:
PointSlot.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('x', 'y'),
              '__init__': <function __main__.PointSlot.__init__(self, x, y)>,
              'x': <member 'x' of 'PointSlot' objects>,
              'y': <member 'y' of 'PointSlot' objects>,
              '__doc__': None})

In [96]:
p.z = 900

In [97]:
p.__dict__

{'x': 10, 'y': 20, 'z': 900}

In [98]:
ps.z = 900

AttributeError: 'PointSlot' object has no attribute 'z'